# Conversion to ONNX
ONNX is a different format for running machine learning models. The ONNX format is much faster on CPU, sometimes 5 times as fast as PyTorch!

While the EAWSW model is designed to be small, accurate and accessible, for some people it's still too much to run...

Hosting the model as a free service for players is an option. An ONNX version of the model allows us to host the model on CPU yet have faster response times! Given that the model is made in a time with chip shortage, running on hardware I already have inside a server is efficient, scalable and cheaper.

An important note is that ONNX doesn't execute logic by itself, and you have to do that yourself, `onnx_model_manager.py` intends to deal with this for us.

In [1]:
%load_ext autoreload
%autoreload 2

from model_utils import train_model, split_data, get_model, set_pretrained_model_dropout, get_dataset
from config import Config
import json
import matplotlib.pyplot as plt
%matplotlib inline
import math
import random
import time
import onnx
import logging
from onnx_model_manager import OnnxModelManager
from onnxruntime.quantization import quantize_dynamic, QuantType
import os
import datasets
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from model_manager import ModelManager

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
saved_model_path = os.path.join("models", "awsw_main")
saved_model_onnx_path = os.path.join("models", "awsw_onnx")
if not os.path.exists(os.path.join(saved_model_path, "special_tokens_map.json")):
    print("Copying config files from huggingface (needed for conversion)... WARNING: this assumes the structure of the model isn't changed!")
    !cd $saved_model_path && git clone https://huggingface.co/$Config.base_model_name
    !cp -n $saved_model_path/$Config.base_model_basename/* $saved_model_path
    !rm -rf $saved_model_path/$Config.base_model_basename
if not os.path.exists(os.path.join(saved_model_onnx_path, "model.onnx")):
    !python3 -m transformers.onnx --model=$saved_model_path --feature=causal-lm --atol=1e-03 $saved_model_onnx_path

In [3]:
def optimize_onnx():
    model_quant = os.path.join(saved_model_onnx_path, "model_quant.onnx")
    if not os.path.exists(model_quant):
        model_fp32 = os.path.join(saved_model_onnx_path, "model.onnx")
        model_opt = os.path.join(saved_model_onnx_path, "model-opt.onnx")
        quantized_model = quantize_dynamic(model_fp32, model_quant, weight_type = QuantType.QInt8)
        #!rm $model_opt
optimize_onnx()

In [4]:
# Tell pytorch to run this model on the GPU.
device_name = "cuda:0" if torch.cuda.is_available() else "cpu"
device_name = 'cpu'
device = torch.device(device_name)

onnx_model_manager = OnnxModelManager(os.path.join(saved_model_onnx_path, "model-opt.onnx"))
onnx_model_manager_quant = OnnxModelManager(os.path.join(saved_model_onnx_path, "model_quant.onnx"))
tokenizer = AutoTokenizer.from_pretrained('EleutherAI/gpt-neo-125M')
model = AutoModelForCausalLM.from_pretrained(saved_model_path)
model.to(device)
model.eval()
model_manager = ModelManager(model=model, tokenizer=tokenizer, device=device)
print(f"Pretrained model loaded on {device_name}")

Pretrained model loaded on cpu


/opt/conda/lib/python3.8/site-packages/torch/nn/modules/module.py:1383: UserWarning: positional arguments and argument "destination" are deprecated. nn.Module.state_dict will not accept them in the future. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(


In [5]:
prompt = "In my dreams, I'm a dragon"
for i in range(2):
    print("ONNX:", onnx_model_manager.say_raw(prompt, do_sample=True))
    print("ONNX (Quantized):", onnx_model_manager_quant.say_raw(prompt, do_sample=True))
    print("PyTorch:", model_manager.say_raw(prompt, 50, 0.7))
    print('-' * 100)

ONNX: In my dreams, I'm a dragoness who lives in the shadows of death."<d>(Not that this would be any different.)<|endoftext|>
ONNX (Quantized): In my dreams, I'm a dragon." (Even that is a common name) a disposition or a disposition orak ________________________________(?)
<?~~~~Meann?t.<?WhatIsIn<?c>?"<?D?>httpd.<?komm??quot???>http from await[ss]<?rit?mal< face>?< face>are[point of interest]< face><p><msg>i? to "true" that we are "true"<p><?t.<?of<A?c?nt?ad<?q??
PyTorch: In my dreams, I'm a dragoness who likes to party, and I'm pretty sure that's what happened to me."<d><scn>cave2<msg>Sb disapproval "It's not like I could change anything now."<p><msg>c "Why not?"<d><scn>cave2<msg>Sb disapproval "Because it's true."<p><msg>c "I'm not sure if I can do that."<d><scn>cave2<msg>Sb disapproval "I suppose we all are, at
----------------------------------------------------------------------------------------------------
ONNX: In my dreams, I'm a dragoness who lives in the shadows of death."

# Testing

We created a few past (for context) + present prompts (player input) and see the different reactions. This way, we can test the models across different iterations.
The first test involves a old prompt to compare the pre-trained model with the one trained on AWSW. Did it manage to store it's data well? Is it able to write down things that have nothing to do with AWSW? (So we know we didn't overfit).

**This test generates boring and repetetive** replies! It's because we use no good sampling algorithm, but it does give us a indication of what the model has learned!

In [6]:
prompts = [
    ('<p><msg>c "Hey Remy!"<d><scn>park2<msg>Ry happy "Hello, [player_name]."', "How are you?"),
    ('<p><msg>c "I was with Lorem today."<d><scn>park2<msg>Ad normal "Very nice."', "What do you think of Lorem?"),
    ('<p><msg>m "In Tatsu park, Adine and I sat down."', "Oh my god, Adine. What is this?"),
    ('<p><msg>m "I sat down on a chair in Anna\'s lab."', "What will we do here?"),
    ('<p><msg>Nm happy "Ah, nice. I like it here."', "Can you swim?"),
]

for (past, prompt) in prompts:
    print(f"Prompt: {prompt}")
    reply = model_manager.say(past, prompt)
    print(f"[Pytorch] Reply: {reply}\n")
    reply = onnx_model_manager.say(past, prompt)
    print(f"[ONNX] Reply: {reply}\n")
    reply = onnx_model_manager_quant.say(past, prompt)
    print(f"[ONNX Quantized] Reply: {reply}\n")
    print("-" * 10)

Prompt: How are you?
[Pytorch] Reply: park2<msg>Ry shy "I'm fine."<p><msg>c "I'm fine, thank you."<d><scn>park2<msg>Ry shy "I see."<p><msg>c "I see."<d><scn>park2<msg>Ry shy "I see."<p

[ONNX] Reply: park2<msg>Ry shy "I'm fine."<p><msg>c "I'm fine, thank you."<d><scn>park2<msg>Ry shy "I see."<p><msg>c "I see."<d><scn>park2<msg>Ry shy "I see."<p><msg>c "I was with Emera today"<d><scn>park2<msg>Ry normal "Very nice"<|endoftext|>

[ONNX Quantized] Reply: r<?><?xx><?>c<msg<>o0<msg>d<?xx><p><msg<? What?>c<msg<? What now?>d<?t<p>d<?d?<?2<?2>c<?S<?>d<?~~~~?<?=]<?<?<?<?d~~~~?<?d~~~~<?d~~~~<?~~~~d<?d<?d~~~~?<?d~~~~<?d~~~~<?d~~~~<d~~~~<?d

----------
Prompt: What do you think of Lorem?
[Pytorch] Reply: park2<msg>Ad normal "I think he's funny."<|endoftext|>

[ONNX] Reply: park2<msg>Ad normal "I think he's funny."<|endoftext|>

[ONNX Quantized] Reply: <p><msg>c "The rest of our lives on the other side of the barriers."<d><p><msg><I>c "The rest of our lives are boring."<d><msg>o "On the other end o

KeyboardInterrupt: 

# Sampling test

This is gonna be interesting!

In [13]:
for (past, prompt) in prompts:
    print(f"Prompt: {prompt}")
    reply = model_manager.say(past, prompt, top_k = 50, top_p = 0.7)
    print(f"[Pytorch] Reply: {reply}\n")
    reply = onnx_model_manager.say(past, prompt, do_sample = True)
    print(f"[ONNX] Reply: {reply}\n")
    reply = onnx_model_manager_quant.say(past, prompt, do_sample = True)
    print(f"[ONNX Quantized] Reply: {reply}\n")
    print("-" * 10)

Prompt: How are you?
[Pytorch] Reply: park2<msg>Ry sad "I'm fine, really. I just wanted to apologize for earlier than best= great= great<d><scn>park2<msg>Ry shy "Well, that's not all that important, are you."<p><msg>c "No, it's not important."<d><scn>

[ONNX] Reply: park2<msg>Ry shy "I guess I am just tired from all my work, huh?"<p><msg>c "I see."<d><scn>park2<msg>Ry shy "You know, I'm glad they're expecting me. I'd be more it you're just another dummy, you know?"<p><msg>c "What do you think will make it?"<d><scn>park2<msg>Ry shy "I think it won't go well."<p><msg>c "I'll have to make do to how things

[ONNX Quantized] Reply: r<?><?xx><?>c<msg<>o0<msg>d<?xx><p><msg<? What?>c<msg<? What now?>d<?t<p>d<?d?<?2<?2>c<?S<?>d<?~~~~?<?=]<?<?<?<?d~~~~?<?d~~~~<?d~~~~<?~~~~d<?d<?d~~~~?<?d~~~~<?d~~~~<?d~~~~<d~~~~<?d

----------
Prompt: What do you think of Lorem?
[Pytorch] Reply: park2<msg>Ad normal "I think he's cute."<|endoftext|>

[ONNX] Reply: park2<msg>Ad normal "He is good looking."<|endofte

KeyboardInterrupt: 

# RP test
Testing out the injected roleplay actions

In [ ]:
test_rps = [
    "Visit Lorem",
    "Meet with Lorem",
    "Visit Adine",
    "Fight",
    "Attack"
]

for rp in test_rps:
    print(f'[Pytorch] {rp} -> {model_manager.say("", rp, top_k = 50, top_p = 0.7)}')
    print(f'[ONNX] {rp} -> {onnx_model_manager.say("", rp, do_sample = True)}')
    print(f'[ONNX Quantized] {rp} -> {onnx_model_manager_quant.say("", rp, do_sample = True)}')
    print("-" * 10)
    
print("Lowercase test")

for rp in test_rps:
    rp = rp[0].lower() + rp[1:]
    print(f'[Pytorch] {rp} -> {model_manager.say("", rp, top_k = 50, top_p = 0.7)}')
    print(f'[ONNX] {rp} -> {onnx_model_manager.say("", rp, do_sample = True)}')
    print(f'[ONNX Quantized] {rp} -> {onnx_model_manager_quant.say("", rp, do_sample = True)}')
    rp = rp.lower()
    print(f'[Pytorch] {rp} -> {model_manager.say("", rp, top_k = 50, top_p = 0.7)}')
    print(f'[ONNX] {rp} -> {onnx_model_manager.say("", rp, do_sample = True)}')
    print(f'[ONNX Quantized] {rp} -> {onnx_model_manager_quant.say("", rp, do_sample = True)}')
    print("-" * 10)